# Testing accuracy of planetary radius predictions

- Useful imports:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, chi2
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

%matplotlib qt 

from planet import Planet
from star import Star
from target_selection import Config

- Useful constants :

In [2]:
MS = 1.989e30  # kg
RS = 6.96342e8  # m
AS = 4.6  # yo
BSsw = 1  # T
LS = 3.826e26  # W

MJ = 1.8986e27  # kg
RJ = 69911e3  # m
wJ = 1.77e-4  # s-1

- Retrieving dataset :

In [3]:
config_param = Config()
data =pd.read_csv(r'/Users/emauduit/Documents/These/target_selection/Programmes/exoplanet.eu_catalog.csv')
data = config_param.param_names(data=data)

- Initializing Jupiter and the Sun:

In [4]:
jup = Planet(
    name="Jupiter",
    mass=1.0,
    radius={"models": config_param.planet_radius_models, "radius": 1.0},
    distance=5.2,
    worb={"star_mass": MS, "worb": 1.0},
    luminosity={
        "models": config_param.planet_luminosity_models,
        "luminosity": np.nan,
        "star_age": 4.6,
    },
    wrot=1.0,
)

sun = Star(
    name="Soleil",
    mass=1.0,
    radius={"models": config_param.star_radius_models, "radius": 1.0},
    age=AS,
    obs_dist=1.0,
    sp_type = 'V'
)
print(sun.luminosity)

0.6977165691451518


- Empty lists for computed/not computed radius and with/without radius expansion : 

In [20]:
radius_data = [] ; mass_data = []
radius_estimate = []
radius_estimate_expansion = []
star_distance = []
temp_eq = []

- Computing :

In [21]:
for target in data.itertuples():
    print(target.pl_name)
    if 'PSR' in target.pl_name :
        print('Warning : {} has been skipped'.format(target.pl_name))
        continue

    if (
        not np.isnan(target.semi_major_axis)
        and (not np.isnan(target.mass) or not np.isnan(target.mass_sini))
        and not np.isnan(target.star_mass)
    ):

        if np.isnan(target.mass):
            planet_mass = target.mass_sini * np.sqrt(4 / 3.0)
        else:
            planet_mass = target.mass
        if not np.isnan(target.eccentricity):
            planet_distance = target.semi_major_axis * (1 - target.eccentricity)
        else:
            planet_distance = target.semi_major_axis

        if np.isnan(target.star_age):
            star_age = 5.2
        else:
            if target.star_age <= 1e-7:
                print("Stellar age is too small (< 100 yr) !!!")
                continue
            elif target.star_age < 0.5:
                star_age = 0.5
            else:
                star_age = target.star_age

        target_radius = np.nan
        radius_data.append(target.radius) ; mass_data.append(planet_mass)
        print(target.radius)
        planet = Planet(
            name=target.pl_name,
            mass=planet_mass,
            radius={"models": config_param.planet_radius_models, "radius": target_radius},
            distance=planet_distance,
            worb={"star_mass": target.star_mass, "worb": np.nan},
            luminosity={
                "models": config_param.planet_luminosity_models,
                "luminosity": np.nan,
                "star_age": star_age,
            },
            detection_method=target.detection_type,
            wrot=1.0,
        )
        star = Star(
            name="star",
            mass=target.star_mass,
            radius={"models": config_param.star_radius_models, "radius": target.star_radius},
            age=star_age,
            obs_dist=target.star_distance,
            sp_type = str(target.star_sp_type)
        )
        star_distance.append(planet.stardist)
        radius_estimate.append(planet.radius)
        print(star.luminosity)
        if np.isnan(target_radius) and config_param.radius_expansion :# and (not np.isnan(target.eccentricity)) :
            Teq = planet.radius_expansion(luminosity=star.luminosity, eccentricity=target.eccentricity)
            temp_eq.append(Teq)
            print('here')
            #planet.radius_expansion()
            radius_estimate_expansion.append(planet.radius)

11 Com b
nan
53.15657000213334
here
11 Oph b
nan
2.171037556098574e-07
here
11 UMi b
nan
10.213705168807667
here
14 And b
nan
23.499637363842567
here
14 Her b
nan
0.41764722707894464
here
14 Her c
nan
0.41764722707894464
here
16 Cyg B b
nan
0.7322291715776864
here
17 Sco b
nan
1.8037206127689798
here
18 Del b
nan
28.13043102985347
here
1RXS 1609 b
1.7
0.15398714862670643
here
1RXS J235133.3+312720 b
nan
0.02910609883423816
here
1SWASP J1407 b
nan
0.41764722707894464
here
24 Boo b
nan
0.6644734161441683
here
24 Sex b
nan
5.199834742106591
here
24 Sex c
nan
5.199834742106591
here
2M 0103-55 (AB) b
nan
0.022614429209997487
here
2M 0219-39 b
1.44
0.0012662392748684374
here
2M 0441+23 b
nan
6.819581837694185e-07
here
2M 1252+2735 b
2M 1253+2734 b
2M 1258+2630 b
2M 2140+16 b
0.92
0.0004685539314397822
here
2M 2206-20 b
1.3
0.0020004963936895107
here
2M 2236+4751 b
nan
0.06790787733093304
here
2M J2126-81 b
nan
0.022614429209997487
here
2M0045+16
2M0117-34
2M0355
2M0437 b
nan
2.39934854472108

In [22]:
radius_data = np.array(radius_data) ; mass_data = np.array(mass_data)
radius_estimate = np.array(radius_estimate)
radius_estimate_expansion = np.array(radius_estimate_expansion)
star_distance = np.array(star_distance)
temp_eq = np.array(temp_eq)

In [23]:
mask_plot = (~np.isnan(radius_data)) #& (star_distance <= 2.)# & (mass_data <= 1.)#& (radius_estimate_expansion <= 10.)

#print("L'expansion moyenne est de :",np.nanmean(radius_estimate_expansion[mask_plot]-radius_estimate[mask_plot]), " RJ.")

In [ ]:
data_model = pd.DataFrame({'radius_m' : radius_data[mask_plot]*RJ, 'radius_pred_m' : radius_estimate[mask_plot]*RJ,'mass_kg' : mass_data[mask_plot]*MJ, 'temp_eq_K':temp_eq[mask_plot]})

data_model.to_csv('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/radius_expansion.csv', index = False)

In [12]:
plt.figure(figsize=(10,7))
plt.plot(np.sort(radius_data[mask_plot]),radius_data[mask_plot][np.argsort(radius_data[mask_plot])], linestyle='dashed', color='tab:blue', label = 'Measured values')
plt.scatter(radius_estimate[mask_plot],radius_data[mask_plot], marker ='x', color = 'tab:orange', label = 'Estimated values')
plt.scatter(radius_estimate_expansion[mask_plot],radius_data[mask_plot], marker = 'x', color='tab:green', label='Estimated values with expansion')
plt.xlabel('R_pred [RJ]')
plt.ylabel('R_measured [RJ]')
plt.title('Evolution of measured radius values compared to predicted values.')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/fit_mass_radius_with_expT.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

## Correlation between predicted and measured values

In [9]:
correl_est , _ = pearsonr(radius_data[mask_plot],radius_estimate[mask_plot])
correl_exp , _ = pearsonr(radius_data[mask_plot],radius_estimate_expansion[mask_plot])

print(correl_est , _)
print(correl_exp , _)
print(np.std(radius_data[mask_plot]))
print(np.std(radius_estimate[mask_plot]))
print(np.std(radius_estimate_expansion[mask_plot]))

0.8393642544027542 0.0
0.8714872489628778 0.0
0.5305153154976004
0.29296872202685087
0.45699018380882306


In [10]:
print(np.min(radius_estimate[mask_plot]),np.max(radius_estimate[mask_plot]))
print(np.min(radius_estimate_expansion[mask_plot]),np.max(radius_estimate_expansion[mask_plot]))

0.08603117776135798 1.0996886248159088
0.061354571797176856 1.847370384747493


In [11]:
print(np.correlate(np.sort(radius_data[mask_plot]),np.sort(radius_data[mask_plot])))
print(chi2(radius_data[mask_plot],radius_data[mask_plot]).value)
print(correl_est)

[1107.83808]


AttributeError: 'rv_frozen' object has no attribute 'value'

In [12]:
plt.plot(radius_data[mask_plot]/radius_data[mask_plot],radius_estimate[mask_plot]/radius_data[mask_plot])
plt.plot(radius_data[mask_plot]/radius_data[mask_plot],radius_estimate_expansion[mask_plot]/radius_data[mask_plot])
plt.grid()
plt.show()

In [13]:
print(radius_estimate_expansion[mask_plot]/radius_data[mask_plot])

[1.06104817 1.09434583 1.92340055 ... 0.94265323 1.05293412 1.25270217]


In [11]:
plt.figure(figsize=(10,7))
plt.plot(np.sort(radius_data[mask_plot]),radius_data[mask_plot][np.argsort(radius_data[mask_plot])], linestyle='dashed', color='tab:blue', label = 'Data')
plt.scatter(radius_data[mask_plot],radius_estimate[mask_plot], marker ='o', color = 'tab:orange', label = 'Estimated values\n correl = {:2f}'.format(correl_est))
plt.scatter(radius_data[mask_plot],radius_estimate_expansion[mask_plot], marker = 'o', color='tab:green', label='Estimated values with expansion \n correl = {:2f}'.format(correl_exp))
plt.xlabel('R$_{measured}$ [RJ]')
plt.ylabel('Planetary radius [RJ]')
plt.title('Evolution of estimated radius values compared to known values.')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/fit_mass_radius_with_expT.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

## Radius as a function of mass :

In [ ]:
mass = np.logspace(-2,1.4,500)
R = np.array([Planet._calculate_radius(models = config_param.planet_radius_models, mass = mass[i]) for i in range(500)])


In [14]:
plt.figure(figsize=(10,7))
#plt.plot(mass,R, linestyle = 'dashed',color = 'black',label='model')
plt.scatter(mass_data[mask_plot],radius_data[mask_plot], marker='+', color='tab:blue', label = 'Measured values')
plt.scatter(mass_data[mask_plot],radius_estimate[mask_plot], marker ='+', color = 'tab:orange', label = 'Estimated values')#\n correl = {:2f}'.format(correl_est))
plt.scatter(mass_data[mask_plot],radius_estimate_expansion[mask_plot], marker = '+', color='tab:green', label='Estimated values with expansion')#\n correl = {:2f}'.format(correl_exp))
plt.xlabel('Planetary mass [MJ]')
plt.ylabel('Planetary radius [RJ]')
plt.title('Evolution of radius values compared to mass.')
plt.xscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/Rp_vs_Mp.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
#plt.plot(,R, linestyle = 'dashed',color = 'black',label='model')
plt.scatter(star_distance[mask_plot],radius_data[mask_plot], marker='o', color='tab:blue', label = 'Data')
plt.scatter(star_distance[mask_plot],radius_estimate[mask_plot], marker ='^', color = 'tab:orange', label = 'Estimated values')
plt.scatter(star_distance[mask_plot],radius_estimate_expansion[mask_plot], marker = '+', color='tab:green', label='Estimated values with expansion\n correl = {:2f}'.format(correl_exp))
plt.xlabel('Star distance [UA]')
plt.ylabel('Planetary radius [RJ]')
plt.title('Evolution of radius values compared to mass.')
plt.xscale('log')
plt.legend()
plt.grid()
plt.show()

## Radius as a function of temperature and semi-major axis

In [10]:
fig,ax = plt.subplots(figsize=(10,7))
ax.scatter(star_distance[mask_plot],temp_eq[mask_plot], c=radius_data[mask_plot], cmap = 'viridis')
plt.xscale('log')
plt.xlabel('Star-planet distance [AU]')
plt.ylabel('Equilibrium temperature [K]')
plt.colorbar(label='Planetary Radius [RJ]')
plt.grid()
plt.show()

RuntimeError: No mappable was found to use for colorbar creation. First define a mappable such as an image (with imshow) or a contour set (with contourf).

In [9]:
fig,ax = plt.subplots(2,2,figsize=(4 * 3, 4 * 3))
#.title('$\Delta$RM as a function of RM, during day/night time')

sc1 = ax[0,0].scatter(star_distance[mask_plot],temp_eq[mask_plot], c=radius_data[mask_plot], cmap = 'viridis')
ax[0,0].set_xlabel('Star-planet distance [AU]')
ax[0,0].set_ylabel('Equilibrium temperature [K]')
ax[0,0].set_xscale('log')
cax1 = inset_axes(
            ax[0,0],
            width='3%',
            height='100%',
            loc='lower left',
            bbox_to_anchor=(1.03, 0., 1, 1),
            bbox_transform=ax[0,0].transAxes,
            borderpad=0,
        )

cbar1 = plt.colorbar(sc1, cax=cax1)
cbar1.set_label('Planetary Radius [RJ]')
ax[0,0].grid()

sc2 = ax[0,1].scatter(star_distance[mask_plot],radius_data[mask_plot], c=temp_eq[mask_plot], cmap = 'viridis')
ax[0,1].set_xlabel('Star-planet distance [AU]')
ax[0,1].set_ylabel('Planetary Radius [RJ]')
ax[0,1].set_xscale('log')
cax2 = inset_axes(
            ax[0,0],
            width='3%',
            height='100%',
            loc='lower left',
            bbox_to_anchor=(1.03, 0., 1, 1),
            bbox_transform=ax[0,1].transAxes,
            borderpad=0,
        )

cbar2 = plt.colorbar(sc2, cax=cax2)
cbar2.set_label('Equilibrium temperature [K]')
ax[0,1].grid()


sc3 = ax[1,0].scatter(temp_eq[mask_plot],radius_data[mask_plot], c=np.log10(star_distance[mask_plot]), cmap = 'viridis')
ax[1,0].set_xlabel('Equilibrium temperature [K]')
ax[1,0].set_ylabel('Planetary Radius [RJ]')
ax[1,0].set_xscale('log')
cax3 = inset_axes(
            ax[0,0],
            width='3%',
            height='100%',
            loc='lower left',
            bbox_to_anchor=(1.03, 0., 1, 1),
            bbox_transform=ax[1,0].transAxes,
            borderpad=0,
        )

cbar3 = plt.colorbar(sc3, cax=cax3)
cbar3.set_label('Star-planet distance [AU]')
ax[1,0].grid()


fig.subplots_adjust(wspace=0.4, hspace=0.2)

## Updated version for radius expansion

In [11]:
def poly_func(coeffs, x) :
    res = 0
    for i,c in enumerate(coeffs[::-1]):
        res +=  c*np.power(x,i)
    return res

### 1 : No radius expansion : new mass-radius relation with polyfit

In [14]:
mass_sorted = np.sort(mass_data[mask_plot]) ; radius_sorted = radius_data[mask_plot][np.argsort(mass_data[mask_plot])]
plt.figure(figsize=(10,7))
plt.plot(mass_sorted,radius_sorted, marker='x', color='tab:blue', label = 'Data')
for i in range(1,12):
    coeffs = np.polyfit(np.log10(mass_sorted),np.log10(radius_sorted), deg = i)
    if i==9 :
        print(coeffs[::-1])
        plt.plot(mass_sorted,10**poly_func(coeffs,np.log10(mass_sorted)), linestyle='dashed',color='tab:red',label='ordre'+str(i))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Planetary mass [MJ]')
plt.ylabel('Planetary radius [RJ]')
plt.title('Evolution of radius with respect to mass.')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/fit_mass_radius.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

[ 0.08488521 -0.03955287 -0.35497347  0.37678815  0.21794621 -0.1517704
 -0.08795222  0.01242521  0.01238428  0.00165453]


- Test new method : polyfit order 9

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(mass_data[mask_plot],radius_data[mask_plot], marker='o', color='tab:blue', label = 'Data')
plt.scatter(mass_data[mask_plot],radius_estimate[mask_plot], marker ='o', color = 'tab:orange', label = 'Estimated values')
plt.xlabel('Planetary mass [MJ]')
plt.ylabel('Planetary radius [RJ]')
plt.title('Evolution of radius values compared to mass.')
plt.xscale('log')
#plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/polyfit_mass_radius_loglin.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

### 2 : Same mass-radius relation as before, but new model for radius expansion

In [24]:
expansion_factor = radius_data[mask_plot]/radius_estimate[mask_plot]

mass_sorted = np.sort(mass_data[mask_plot]) ; expansion_sorted = expansion_factor[np.argsort(mass_data[mask_plot])]
radius_sorted = radius_data[mask_plot][np.argsort(mass_data[mask_plot])]
#Teq_sorted = expansion_factor[np.argsort(temp_eq[mask_plot])]

plt.figure(figsize=(10,7))
plt.scatter(mass_sorted,expansion_sorted, marker='o', color='tab:blue', label = 'Data')
for i in range(1,15):
    coeffs = np.polyfit(np.log10(mass_sorted),np.log10(expansion_sorted), deg = i)
    correl, _ = pearsonr(expansion_sorted, 10**poly_func(coeffs,np.log10(mass_sorted)))
    if i==9 :
        print(coeffs[::-1])
        plt.plot(mass_sorted,10**poly_func(coeffs,np.log10(mass_sorted)), color='tab:red',linestyle='dashed',label='Ordre {}, c={:1.3f}'.format(i,correl))

plt.xlabel('Planetary mass [MJ]')
plt.ylabel('Radius expansion [dimensionless]')
plt.title('Evolution of radius expansion with respect to mass.')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/expansion_vs_mass.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

In [16]:
plt.figure(figsize=(10,7))
plt.scatter(star_distance[mask_plot],expansion_factor, marker='o', color='tab:blue', label = 'Data')
plt.xlabel('Star-planet distance [UA]')
plt.ylabel('Radius expansion [dimensionless]')
plt.title('Evolution of radius expansion with respect to star-planet distance.')
plt.xlim(1e-3,10)
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/expansion_vs_stardist.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

- test polyfit of order 9 :

In [17]:
plt.figure(figsize=(10,7))
plt.scatter(mass_sorted,radius_sorted, marker='o', color='tab:blue', label = 'Data')
plt.scatter(mass_data[mask_plot],radius_estimate_expansion[mask_plot], marker = 'o', color='tab:red', label='Estimated with expansion')
plt.xlabel('Planetary mass [MJ]')
plt.ylabel('Radius [RJ]]')
plt.title('Evolution of radius with expansion with respect to mass.')
plt.xscale('log')
#plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/radius_exp_vs_mass_loglin.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

### Fit de Expansion(Teq)

In [25]:
#expansion_factor = radius_data[mask_plot]/radius_estimate[mask_plot]
mask_fit = (expansion_factor >= 0.8) & (expansion_factor <= 1.9) &  (temp_eq[mask_plot] >= 400)

#expansion_sorted = np.sort(expansion_factor[mask_fit])
#Teq_sorted = temp_eq[np.argsort(expansion_factor[mask_fit])]

coeffs = np.polyfit(np.log10(temp_eq[mask_plot][mask_fit]),expansion_factor[mask_fit], deg = 2)
correl, _ = pearsonr(expansion_factor[mask_fit], poly_func(coeffs,np.log10(temp_eq[mask_plot][mask_fit])))

In [ ]:
print(coeffs[::-1])

In [35]:
plt.figure(figsize=(10,7))
plt.scatter(temp_eq[mask_plot],expansion_factor, marker='o', color='tab:blue', label = 'Data')
plt.scatter(temp_eq[mask_plot][mask_fit], poly_func(coeffs,np.log10(temp_eq[mask_plot][mask_fit])), marker = 'o', color='tab:red', label='fit_ordre 2')
plt.hlines(1.9, xmin = 0.2,xmax =6000,linestyles='dashed', color='black')
plt.hlines(0.8, xmin = 0.2,xmax =6000,linestyles='dashed', color='black')
plt.vlines(400, ymin=0,ymax=6,linestyles='dashed', color='black')
plt.xlabel('Temperature [K]')
plt.ylabel('Expansion factor')
plt.title('Evolution of expansion with respect to temperature.')
plt.xlim(0.2,6000)
plt.ylim(0,6)
plt.xscale('log')
#plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/exp_vs_Teq.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

## Fit de R_meas(R_pred * exp)

In [36]:
radius_estimate_expansion_old = np.copy(radius_estimate_expansion)

In [37]:
mask_fit_2 = (~np.isnan(radius_data)) & (~np.isnan(radius_estimate_expansion_old))
coeffs_2 = np.polyfit(np.log10(radius_estimate_expansion_old[mask_fit_2]),np.log10(radius_data[mask_fit_2]), deg = 4)
correl_2, _ = pearsonr(radius_data[mask_fit_2], 10**poly_func(coeffs_2,np.log10(radius_estimate_expansion_old[mask_fit_2])))
print(coeffs_2[::-1])

[ 0.02518801  0.99289611 -1.50999601 -3.14340725 -1.59546504]


In [41]:
plt.figure(figsize=(10,7))
plt.scatter(radius_estimate_expansion[mask_fit_2],radius_data[mask_fit_2], marker='o', color='tab:blue', label = 'Data')
#plt.scatter(radius_estimate_expansion[mask_fit_2],radius_estimate_expansion[mask_fit_2]*(10**poly_func(coeffs_2,np.log10(radius_estimate_expansion[mask_fit_2]))), marker='+', color='tab:orange')
plt.scatter(radius_estimate_expansion[mask_fit_2],10**poly_func(coeffs_2,np.log10(radius_estimate_expansion[mask_fit_2])), marker='o', color='tab:red',label='Fit, order 4')
plt.plot([0.08,2.5],[0.08,2.5], linestyle='dashed', color='black')
plt.xlabel('Radius estimate with expansion [RJ]')
plt.ylabel('Radius measured [RJ]')
plt.title('Evolution of radius measured with respected to predicted radius with expansion')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/Rpred_vs_Rmeas.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

In [39]:
plt.figure(figsize=(10,7))
plt.scatter(radius_estimate_expansion_old[mask_fit_2],radius_data[mask_fit_2], marker='o', color='tab:blue', label = 'Data', alpha=0.5)
plt.scatter(10**poly_func(coeffs_2,np.log10(radius_estimate_expansion_old[mask_fit_2])),radius_data[mask_fit_2], marker='o', color='tab:red',label='Fit, order 4',alpha=0.5)
plt.scatter(radius_estimate_expansion[mask_fit_2],radius_data[mask_fit_2], marker='+', color='tab:orange')
plt.plot([0.08,2.5],[0.08,2.5], linestyle='dashed', color='black')
plt.xlabel('Radius estimate with expansion [RJ]')
plt.ylabel('Radius measured [RJ]')
plt.title('Evolution of radius measured with respected to predicted radius with expansion')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/Rpred_vs_Rmeas.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

In [40]:
plt.figure(figsize=(10,7))
plt.scatter(radius_estimate_expansion[mask_fit_2],10**poly_func(coeffs_2,np.log10(radius_estimate_expansion[mask_fit_2]))/radius_estimate_expansion[mask_fit_2], marker='o', color='tab:blue', label = 'Data')
#plt.plot([0.08,2.5],[0.08,2.5], linestyle='dashed', color='black')
plt.xlabel('Rp * E')
plt.ylabel('Rp* E corrigé')
plt.title('Evolution of radius measured with respected to predicted radius with expansion')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()
#plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/Rpred_vs_Rmeas.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

## Recomputing constants for the M_p-R_p relation from Griessmeier, 2017, A&A

In [ ]:
def mass_radius_relation(Mp,rho0,Mmax) :
    RJ = 69911e3
    res = np.power((4 / 3) * np.pi * rho0, -1.0 / 3) * np.power(Mp * 1.8986e27, 1.0 / 3) / (1 + np.power(Mp / Mmax, 2.0 / 3))
    return res / RJ

In [ ]:
mask_plot = (~np.isnan(radius_data)) #& (mass_data >= 0.01) & (mass_data <= 10.)
mass_sorted = np.sort(mass_data[mask_plot])
radius_sorted = radius_data[mask_plot][np.argsort(mass_data[mask_plot])]

In [ ]:
from scipy.optimize import curve_fit

popt, pcov = curve_fit(mass_radius_relation,mass_sorted,radius_sorted, p0=[394,3.16])

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(mass_sorted,radius_sorted, marker='o', color='tab:blue', label = 'Data')
plt.scatter(mass_sorted,mass_radius_relation(mass_sorted,*popt), marker = 'x', color='tab:red', label='New params, p_0 = {:3f}, M_max = {:3f}'.format(*popt))
plt.scatter(mass_sorted,mass_radius_relation(mass_sorted,394,3.16), marker = 'x', color='tab:orange', label='Old params, p_0 = {394 kg/m3}, M_max = {3.16 MJ}')
plt.xlabel('Planetary mass [MJ]')
plt.ylabel('Radius [RJ]]')
plt.title('Evolution of radius with expansion with respect to mass.')
plt.xscale('log')
#plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/new_fit.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()